## ChatGPT

This is just some messing around.

Docs:

* how to upload images: https://platform.openai.com/docs/guides/vision
* types of models: https://stackoverflow.com/questions/75774873/openai-api-error-this-is-a-chat-model-and-not-supported-in-the-v1-completions
* token limits: https://platform.openai.com/settings/organization/limits
* models: https://platform.openai.com/docs/models/model-endpoint-compatibility

In [4]:
#!pip install openai

In [51]:
import openai
import base64
from openai import OpenAI
from PIL import Image
import numpy as np
import json

In [27]:
def load_image(image_path):
    img = Image.open(image_path).convert('RGB')
    new_size = np.round(np.array(img.size)*0.5).astype('int')
    img = img.resize(new_size, Image.Resampling.LANCZOS)
    #img = np.array(img)
    #with open(image_path, "rb") as image_file:
    img.save('/Users/jnaiman/Downloads/tmp/tmp_img.png')
    with open('/Users/jnaiman/Downloads/tmp/tmp_img.png','rb') as image_file:
        #return base64.b64encode(img).decode("utf-8")
        return base64.b64encode(image_file.read()).decode("utf-8")

In [53]:
# get image
iFile = 1
image_path = '/Users/jnaiman/Downloads/data_full_v2/Picture'+str(int(iFile))+'.png'
encoded_image = load_image(image_path)

In [55]:
# get questions
with open('/Users/jnaiman/Downloads/data_full_v2/Picture'+str(int(iFile))+'_qa.json','r') as f:
    j = json.loads(f.read())
    j = json.loads(j)

In [120]:
qa = []

for k,v in j['Level 1']['Figure-level questions'].items():
    out = {'Q':v['Q'], 'A':v['A'], 'Level':'Level 1', 'type':'Figure-level questions', 'Response':""}
    qa.append(out)

# what kinds?
types = ['(words + list)', '(words)']

# get uniques
keys_tmp = list(j['Level 1']['Plot-level questions'].keys())
keys = []
for k in keys_tmp:
    if '(' in k:
        k = k.split('(')[0].rstrip()
    keys.append(k)

for k in keys:
    v = ''
    kk = ''
    for t in types:
        if k + " " + t in j['Level 1']['Plot-level questions']:
            v = j['Level 1']['Plot-level questions'][k + " " + t]
            #kk = k + " " + t
            break
    if v == '':
        v = j['Level 1']['Plot-level questions'][k]
        #kk = k
    if 'A' in v: # no plot
        if type(v['A']) == type({}):
            ans = list(v['A'].values())[0]
        else:
            ans = v['A']
        if type(v['Q']) == type({}):
            que = list(v['Q'].values())[0]
        else:
            que = v['Q']
    else: # plotX
        for kk,vv in v.items():
            ans = vv['A']
            while type(ans) == type({}):
                ans = list(ans.values())[0]
            que = vv['Q']
    out = {'Q':que, 'A':ans, 'Level':'Level 1', 'type':'Plot-level questions', 'Response':""}
    # if there is a plot
    if 'plot' in list(v.keys())[0]:
        out['plot number'] = list(v.keys())[0]
        #import sys; sys.exit()
    qa.append(out)

# # plot by plot
# for k,v in j['Level 1']['Plot-level questions'].items():
#     if '(plot numbers)': # no by numbers
#         pass
qa

[{'Q': 'How many panels are in this figure? You are a helpful assistant, please format the output as a json as {"nrows":"", "ncols":""} to store the number of rows and columns.',
  'A': {'nrows': 1, 'ncols': 1},
  'Level': 'Level 1',
  'type': 'Figure-level questions',
  'Response': ''},
 {'Q': 'Assuming this is a figure made with matplotlib in Python, what is the plot style used?  Examples of plotting styles are "classic" or "ggplot". You are a helpful assistant, please format the output as a json as {"plot style":""} to store the matplotlib plotting style used in the figure.',
  'A': {'plot style': 'bmh'},
  'Level': 'Level 1',
  'type': 'Figure-level questions',
  'Response': ''},
 {'Q': 'Assuming this is a figure made with matplotlib in Python, what is the colormap that was used?  Examples of matplotlib colormaps are "rainbow" or "Reds". You are a helpful assistant, please format the output as a json as {"colormap":""} to store the matplotlib colormap used in the figure.',
  'A': {

In [121]:
#qa

In [122]:
#j['Level 1']['Plot-level questions']['nlines (plot numbers)'].keys()

In [123]:
#vv['A']

In [ ]:
# setup
with open('/Users/jnaiman/.openai/key.txt','r') as f:
    api_key = f.read()

client = OpenAI(
  api_key=api_key.strip(),  # this is also the default, it can be omitted
)

#############

# Step 3: Ask a series of questions
# questions = [
#     "What is happening in this image?",
#     "What is the main object in this image?",
#     "Can you describe the colors in this image?",
#     "Does this image contain any text? If so, what does it say?"
# ]

responses = []

for question_list in qa:
    question = question_list['Q']
    # Prepare the API request
    prompt = f"I am going to show you an image. Here is the image: [Image: {encoded_image}]. Now, {question}"
    
    # Send the request to the GPT-4o API
    response = client.chat.completions.create(
        #model="gpt-4",
        #model="gpt-4o",
        model ="gpt-4o-mini",
        #model ="gpt-3.5-turbo",
        #model="gpt-3.5-turbo-instruct",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that can analyze images."},
            {"role":"user", "content": [
                {
                  "type": "text",
                  "text": prompt
                },
                {
                  "type": "image_url",
                  "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded_image}"
                  }
                }
            ]
            }
        ]
    )
    
    # Get the response from the API
    #answer = response.choices[0].text #response.choices[0].message["content"]
    answer = response.choices[0].message.content
    #responses.append((question, answer))
    # format answer
    answer_format = answer.replace("```json\n",'').replace("\n```",'')
    question_list['Response'] = json.loads(answer_format)

# Step 4: Handle the responses
# for question, answer in responses:
#     print(f"Q: {question}\nA: {answer}\n")

In [67]:
#qa

In [ ]:
for level in qa:
    print('Q:', level['Q'])
    #print('ChatGPT A:', json.loads(level['Response'].replace("```json\n",'').replace("\n```",'')))
    print('ChatGPT A:', level['Response'])
    print('Real A:   ', level['A'])
    print('')

In [47]:
# client.chat.completions.create?

In [48]:
# payload = {
#   "model": "gpt-4o-mini",
#   "messages": [
#     {
#       "role": "user",
#       "content": [
#         {
#           "type": "text",
#           "text": "What’s in this image?"
#         },
#         {
#           "type": "image_url",
#           "image_url": {
#             "url": f"data:image/jpeg;base64,{base64_image}"
#           }
#         }
#       ]
#     }
#   ],
#   "max_tokens": 300
# }